In [1]:
import pandas as pd

In [2]:
import fasttext

In [3]:
data = pd.read_csv('text_emotion.csv')

In [4]:
data

,tweet_id,sentiment,author,content
0,1956967341,empty,xoshayzers,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,wannamama,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,coolfunky,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,czareaquino,wants to hang out with friends SOON!
4,1956968416,neutral,xkilljoyx,@dannycastillo We want to trade with someone w...
...,...,...,...,...
39995,1753918954,neutral,showMe_Heaven,@JohnLloydTaylor
39996,1753919001,love,drapeaux,Happy Mothers Day All my love
39997,1753919005,love,JenniRox,Happy Mother's Day to all the mommies out ther...
39998,1753919043,happiness,ipdaman1,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [6]:
data = data.drop(labels=['tweet_id', 'author'], axis=1)

In [7]:
data

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...
...,...,...
39995,neutral,@JohnLloydTaylor
39996,love,Happy Mothers Day All my love
39997,love,Happy Mother's Day to all the mommies out ther...
39998,happiness,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [8]:
from sklearn.preprocessing import LabelEncoder

labelen = LabelEncoder()

In [9]:
data['sentiment'] = labelen.fit_transform(data['sentiment'])

In [11]:
data

,sentiment,content
0,2,@tiffanylue i know i was listenin to bad habi...
1,10,Layin n bed with a headache ughhhh...waitin o...
2,10,Funeral ceremony...gloomy friday...
3,3,wants to hang out with friends SOON!
4,8,@dannycastillo We want to trade with someone w...
...,...,...
39995,8,@JohnLloydTaylor
39996,7,Happy Mothers Day All my love
39997,7,Happy Mother's Day to all the mommies out ther...
39998,5,@niariley WASSUP BEAUTIFUL!!! FOLLOW ME!! PEE...


In [12]:
data.to_csv('new_sentiment.csv')

In [16]:
import csv  
import re

train = open('tweets.train','w')  
test = open('tweets.valid','w')  
with open('new_sentiment.csv', mode='r') as csv_file:  
    csv_reader = csv.DictReader(csv_file)
    line = 0
    for row in csv_reader:
        # Clean the training data
        # First we lower case the text
        text = row['content'].lower()
        # remove links
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
        #Remove usernames
        text = re.sub('@[^\s]+','', text)
        # replace hashtags by just words
        text = re.sub(r'#([^\s]+)', r'\1', text)
        #correct all multiple white spaces to a single white space
        text = re.sub('[\s]+', ' ', text)
        # Additional clean up : removing words less than 3 chars, and remove space at the beginning and teh end
        text = re.sub(r'\W*\b\w{1,3}\b', '', text)
        text = text.strip()
        line = line + 1
        # Split data into train and validation
        if line%16 == 0:
            print(f'__label__{row["sentiment"]} {text}', file=test)
        else:
            print(f'__label__{row["sentiment"]} {text}', file=train)

In [18]:
#BASIC MODEL
model1 = fasttext.train_supervised('tweets.train')

In [19]:
model1.predict('tweets.valid')

(('__label__8',), array([0.92535073]))

In [20]:
model1.predict('completing facebook quizzes this time morning... especially friday morning')

(('__label__8',), array([0.36366096]))

In [21]:
model1.predict('need held... gnite.')

(('__label__8',), array([0.60163003]))

In [22]:
model1.predict('little foster kittens getting...chubba winston going missed when they amazing homes')

(('__label__7',), array([0.24965554]))

In [24]:
model1.test('tweets.valid')

(2383, 0.3424255140579102, 0.3424255140579102)

In [26]:
# INCREASING EPOCHS

model2 = fasttext.train_supervised('tweets.train',epoch=10)

In [27]:
model2.predict('completing facebook quizzes this time morning... especially friday morning')

(('__label__8',), array([0.36841801]))

In [28]:
model2.test('tweets.valid')

(2383, 0.3394880402853546, 0.3394880402853546)

In [29]:
# INCREASING EPOCHS, with learning rate

model2 = fasttext.train_supervised('tweets.train',epoch=10, lr=0.01)

In [30]:
model2.test('tweets.valid')

(2383, 0.29458665547629037, 0.29458665547629037)

In [31]:
model4 = fasttext.train_supervised('tweets.train', epoch=25, lr=0.001)

In [32]:
model4.test('tweets.valid')

(2383, 0.21275702895509863, 0.21275702895509863)

In [34]:
model4.predict('completing facebook quizzes this time morning... especially friday morning')

(('__label__8',), array([0.1367474]))

In [43]:
model4.predict('wanted story acorn tonight glen beck missed')

(('__label__8',), array([0.13341016]))

In [41]:
# MODEL %

In [53]:
model4 = fasttext.train_supervised('tweets.train', epoch=30, lr=0.0001, loss='ova')

In [54]:
model4.test('tweets.valid')

(2383, 0.21275702895509863, 0.21275702895509863)

In [55]:
model4 = fasttext.train_supervised('tweets.train', epoch=30, lr=0.0001, loss='ova', bucket=50000)

In [56]:
model4.test('tweets.valid')

(2383, 0.21275702895509863, 0.21275702895509863)

In [57]:
# MODEL 6

In [60]:
model6 = fasttext.train_supervised('tweets.train', epoch=25, lr=0.001, loss='ova', wordNgrams=4)

In [61]:
model6.test('tweets.valid')

(2383, 0.21275702895509863, 0.21275702895509863)

In [ ]:
model7 = fasttext.train_supervised('tweets.train', epoch=25, lr=0.0001, loss='ova', wordNgrams=4)

In [ ]:
model7.test('tweets.valid')